In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 8.8 MB/s eta 0:00:00


In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from IPython import display
import plotly as plotly
%matplotlib inline



import calendar
import datetime as dt
from datetime import timedelta, date
from pprint import pprint

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error


import joblib
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

from scipy.stats import randint as sp_randint
from time import time
import pickle

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
random_state = 123

In [ ]:
# importing libraries
import os
import pandas as pd
import numpy as np
import glob
import psycopg2
import sqlalchemy
import shutil
import glob
import warnings
from __future__ import absolute_import, division, print_function, unicode_literals
import seaborn as sns

import matplotlib as mpl
import matplotlib.pyplot as plt

from datetime import datetime

import plotly.express as px


warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('df_for_ml.csv')
df

,Time,system_kW
0,2021-01-02 19:14:00,9.866
1,2021-01-02 19:15:00,9.865
2,2021-01-02 19:16:00,11.219
3,2021-01-02 19:17:00,11.194
4,2021-01-02 19:18:00,11.234
...,...,...
1446597,2021-01-02 19:09:00,11.165
1446598,2021-01-02 19:10:00,11.185
1446599,2021-01-02 19:11:00,11.190
1446600,2021-01-02 19:12:00,10.467


In [ ]:
df.dropna(inplace=True)
df

,Time,system_kW
0,2021-01-02 19:14:00,9.866
1,2021-01-02 19:15:00,9.865
2,2021-01-02 19:16:00,11.219
3,2021-01-02 19:17:00,11.194
4,2021-01-02 19:18:00,11.234
...,...,...
1446597,2021-01-02 19:09:00,11.165
1446598,2021-01-02 19:10:00,11.185
1446599,2021-01-02 19:11:00,11.190
1446600,2021-01-02 19:12:00,10.467


In [ ]:
# split time column into two columns
df['time1'] = df['Time'].str[:16]
df['time2'] = df['Time'].str[16:]

# drop original time column
# df.drop('time', axis=1, inplace=True)
df

,Time,system_kW,time1,time2
0,2021-01-02 19:14:00,9.866,2021-01-02 19:14,:00
1,2021-01-02 19:15:00,9.865,2021-01-02 19:15,:00
2,2021-01-02 19:16:00,11.219,2021-01-02 19:16,:00
3,2021-01-02 19:17:00,11.194,2021-01-02 19:17,:00
4,2021-01-02 19:18:00,11.234,2021-01-02 19:18,:00
...,...,...,...,...
1446597,2021-01-02 19:09:00,11.165,2021-01-02 19:09,:00
1446598,2021-01-02 19:10:00,11.185,2021-01-02 19:10,:00
1446599,2021-01-02 19:11:00,11.190,2021-01-02 19:11,:00
1446600,2021-01-02 19:12:00,10.467,2021-01-02 19:12,:00


In [ ]:
df1 = pd.to_datetime(df['time1'], format="%d/%m/%Y %H:%M", errors='coerce')
df2 = pd.to_datetime(df['time1'], format="%m/%d/%Y %H:%M", errors='coerce')
df3 = pd.to_datetime(df['time1'], format="%Y-%m-%d %H:%M", errors='coerce')

df4 = df1.combine_first(df2)
df['Time'] = df4.combine_first(df3)

In [ ]:
df

,Time,system_kW,time1,time2
0,2021-01-02 19:14:00,9.866,2021-01-02 19:14,:00
1,2021-01-02 19:15:00,9.865,2021-01-02 19:15,:00
2,2021-01-02 19:16:00,11.219,2021-01-02 19:16,:00
3,2021-01-02 19:17:00,11.194,2021-01-02 19:17,:00
4,2021-01-02 19:18:00,11.234,2021-01-02 19:18,:00
...,...,...,...,...
1446597,2021-01-02 19:09:00,11.165,2021-01-02 19:09,:00
1446598,2021-01-02 19:10:00,11.185,2021-01-02 19:10,:00
1446599,2021-01-02 19:11:00,11.190,2021-01-02 19:11,:00
1446600,2021-01-02 19:12:00,10.467,2021-01-02 19:12,:00


In [ ]:
# from datetime import datetime

# # example datetime values
# datetime_values = [
#     "11/01/2021 05:53",
#     "11/01/2021 05:54",
#     "11/01/2021 05:55",
#     "2023-02-28 19:51",
#     "2023-02-28 19:52",
#     "16/02/2021 19:14",
#     "7/15/2021 19:14"
# ]

# # define the output format
# output_format = "%Y-%m-%d %H:%M:%S"

# # loop through each datetime value and convert it to the desired format
# formatted_datetime_values = []
# for datetime_value in datetime_values:
#     if "/" in datetime_value:
#         if datetime_value.count("/") == 2:
#             input_format = "%d/%m/%Y %H:%M"
#         else:
#             input_format = "%d/%m/%Y %H:%M"
#     else:
#         input_format = "%Y-%m-%d %H:%M"
#     formatted_datetime = datetime.strptime(datetime_value, input_format)
#     formatted_datetime_values.append(formatted_datetime.strftime(output_format))

# # print the formatted datetime values
# print(formatted_datetime_values)


In [ ]:
df['Time'] =  pd.to_datetime(df['Time'])
# df['Time'] = df['Time'].dt.tz_convert(None)

# df

# df2

# dfs = df.append([df2])
# dfs

# dfs.drop_duplicates(subset=['Time'], keep='last', inplace=True)

# dfs.to_csv('EHM21120502-2022-11-08T11-19-29+0100-5min.csv',index=False)

In [ ]:
# from datetime import datetime
# import pandas as pd

# def unix_or_dt(a):
#     my_format = '%Y-%m-%d %H:%M:%S'
#     try:
#         return datetime.strptime(str(a), '%Y-%m-%d %H:%M:%S').strftime(my_format)
#     except:
#         return datetime.strptime(str(a), '%d/%m/%Y %H:%M').strftime(my_format)


# df['Time'] = df['Time'].apply(lambda row: unix_or_dt(row))

# df

In [ ]:
df.drop_duplicates(subset=['Time'], keep='last', inplace=True)

In [ ]:
# df.to_csv('df.csv', index=False)
# df['system_kW'] = df.P001_kW + df.P002_kW + df.P003_kW + df.P004_kW + df.P005_kW + df.P006_kW + df.P007_kW +df.P008_kW + df.P009_kW

In [ ]:
df = df[['Time','system_kW']]
# df_for_ML.to_csv('df_for_ml.csv', index=False)

In [ ]:
def time_adjust(df):
    df = df.reset_index()
    df['Time'] = df['Time'].astype(str)
    df['Time']= pd.to_datetime(df['Time'])
    df.sort_values(by='Time', inplace=True)
    return df

def time_indexing(df):
    df = df.set_index('Time')
    df.index = pd.to_datetime(df.index)
    return df

In [ ]:
df.sort_values('Time',ascending=True, inplace=True)

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df.drop(columns=['index'],inplace=True)

In [ ]:
df.dropna(inplace=True)

In [ ]:
df_i = df.set_index('Time')

In [ ]:
df_i

,system_kW
Time,
2020-01-01 19:14:00,9.877
2020-01-01 19:15:00,9.821
2020-01-01 19:16:00,9.831
2020-01-01 19:17:00,9.810
2020-01-01 19:18:00,9.808
...,...
2022-12-31 23:55:00,9.565
2022-12-31 23:56:00,9.588
2022-12-31 23:57:00,9.549


In [ ]:
df_i.loc['2022']

,system_kW
Time,
2022-01-01 00:00:00,10.426
2022-01-01 00:01:00,10.419
2022-01-01 00:02:00,10.385
2022-01-01 00:03:00,10.594
2022-01-01 00:04:00,10.678
...,...
2022-12-31 23:55:00,9.565
2022-12-31 23:56:00,9.588
2022-12-31 23:57:00,9.549


In [ ]:
# data re-sampled based on an hour
df_by_hr_max = df.resample('H', on='Time').system_kW.max()

df_by_hr_max = pd.DataFrame(df_by_hr_max).reset_index()

df_by_hr_max.rename(columns = {'system_kW':'Max_Load'}, inplace = True)

import pandas as pd

# Get week number, with pd.DateOffset used to start the week on Sunday
week_date_values = df_by_hr_max["Time"].dt.isocalendar()
df_by_hr_max["week_date"] = (df_by_hr_max['Time'] - pd.to_timedelta(df_by_hr_max['Time'].dt.dayofweek, unit='d') + pd.DateOffset(days=-1)).dt.strftime("%Y-%m-%d")

# Combine with "(week n)" values
df_by_hr_max["week_date"] = df_by_hr_max["week_date"] + " " + "(week " + week_date_values["week"].astype(str) + ")"
df_by_hr_max

# data re-sampled based on an hour
df_by_hr_min = df.resample('H', on='Time').system_kW.min()

df_by_hr_min = pd.DataFrame(df_by_hr_min).reset_index()

df_by_hr_min.rename(columns = {'system_kW':'Min_Load'}, inplace = True)

import pandas as pd

# Get week number, with pd.DateOffset used to start the week on Sunday
week_date_values = df_by_hr_min["Time"].dt.isocalendar()
df_by_hr_min["week_date"] = (df_by_hr_min['Time'] - pd.to_timedelta(df_by_hr_min['Time'].dt.dayofweek, unit='d') + pd.DateOffset(days=-1)).dt.strftime("%Y-%m-%d")

# Combine with "(week n)" values
df_by_hr_min["week_date"] = df_by_hr_min["week_date"] + " " + "(week " + week_date_values["week"].astype(str) + ")"
Min_Load = df_by_hr_min.Min_Load.to_list()

# data re-sampled based on an hour
df_by_hr_mean = df.resample('H', on='Time').system_kW.mean()

df_by_hr_mean = pd.DataFrame(df_by_hr_mean).reset_index()

df_by_hr_mean.rename(columns = {'system_kW':'Mean_Load'}, inplace = True)

import pandas as pd

# Get week number, with pd.DateOffset used to start the week on Sunday
week_date_values = df_by_hr_mean["Time"].dt.isocalendar()
df_by_hr_mean["week_date"] = (df_by_hr_mean['Time'] - pd.to_timedelta(df_by_hr_mean['Time'].dt.dayofweek, unit='d') + pd.DateOffset(days=-1)).dt.strftime("%Y-%m-%d")

# Combine with "(week n)" values
df_by_hr_mean["week_date"] = df_by_hr_mean["week_date"] + " " + "(week " + week_date_values["week"].astype(str) + ")"
Mean_Load = df_by_hr_mean.Mean_Load.to_list()

df_max_min_mean = df_by_hr_max.assign(Min_Load=Min_Load,Mean_Load=Mean_Load)

df_hr_max_min_mean = df_max_min_mean[['Time','week_date','Max_Load','Min_Load','Mean_Load']]

df_hr_max_min_mean

,Time,week_date,Max_Load,Min_Load,Mean_Load
0,2020-01-01 19:00:00,2019-12-29 (week 1),9.930,9.808,9.855870
1,2020-01-01 20:00:00,2019-12-29 (week 1),9.870,9.727,9.803250
2,2020-01-01 21:00:00,2019-12-29 (week 1),11.638,9.728,11.166733
3,2020-01-01 22:00:00,2019-12-29 (week 1),11.503,11.320,11.400533
4,2020-01-01 23:00:00,2019-12-29 (week 1),11.419,8.853,10.381850
...,...,...,...,...,...
26280,2022-12-31 19:00:00,2022-12-25 (week 52),11.372,10.022,10.470167
26281,2022-12-31 20:00:00,2022-12-25 (week 52),11.178,9.939,10.393550
26282,2022-12-31 21:00:00,2022-12-25 (week 52),11.119,9.764,10.206517
26283,2022-12-31 22:00:00,2022-12-25 (week 52),10.888,9.574,10.092483


In [ ]:
temp = pd.read_csv("temptemp.csv")
temp

,YEAR,MO,DY,HR,T2M,T2MDEW,T2MWET
0,2021,1,1,0,26.11,25.01,25.56
1,2021,1,1,1,25.95,24.92,25.44
2,2021,1,1,2,25.80,24.86,25.33
3,2021,1,1,3,25.65,24.77,25.21
4,2021,1,1,4,25.53,24.68,25.10
...,...,...,...,...,...,...,...
17515,2022,12,31,19,24.83,16.94,20.89
17516,2022,12,31,20,24.73,16.83,20.78
17517,2022,12,31,21,24.69,16.70,20.69
17518,2022,12,31,22,24.73,16.55,20.64


In [ ]:
temp = temp[temp['T2M']>0]
temp

,YEAR,MO,DY,HR,T2M,T2MDEW,T2MWET
0,2021,1,1,0,26.11,25.01,25.56
1,2021,1,1,1,25.95,24.92,25.44
2,2021,1,1,2,25.80,24.86,25.33
3,2021,1,1,3,25.65,24.77,25.21
4,2021,1,1,4,25.53,24.68,25.10
...,...,...,...,...,...,...,...
17515,2022,12,31,19,24.83,16.94,20.89
17516,2022,12,31,20,24.73,16.83,20.78
17517,2022,12,31,21,24.69,16.70,20.69
17518,2022,12,31,22,24.73,16.55,20.64


In [ ]:


# create a new column for datetime as string with desired format using YEAR, MO, DY, and HR columns
temp['datetime'] = temp['YEAR'].astype(str) + '-' + temp['MO'].astype(str).str.zfill(2) + '-' + temp['DY'].astype(str).str.zfill(2) + ' ' + temp['HR'].astype(str).str.zfill(2)

# drop the individual YEAR, MO, DY, and HR columns
temp.drop(['YEAR', 'MO', 'DY', 'HR'], axis=1, inplace=True)

temp


,T2M,T2MDEW,T2MWET,datetime
0,26.11,25.01,25.56,2021-01-01 00
1,25.95,24.92,25.44,2021-01-01 01
2,25.80,24.86,25.33,2021-01-01 02
3,25.65,24.77,25.21,2021-01-01 03
4,25.53,24.68,25.10,2021-01-01 04
...,...,...,...,...
17515,24.83,16.94,20.89,2022-12-31 19
17516,24.73,16.83,20.78,2022-12-31 20
17517,24.69,16.70,20.69,2022-12-31 21
17518,24.73,16.55,20.64,2022-12-31 22


In [ ]:
# temp['datetime'] =
temp['datetime'] =  pd.to_datetime(temp['datetime'])

In [ ]:
temp

,T2M,T2MDEW,T2MWET,datetime
0,26.11,25.01,25.56,2021-01-01 00:00:00
1,25.95,24.92,25.44,2021-01-01 01:00:00
2,25.80,24.86,25.33,2021-01-01 02:00:00
3,25.65,24.77,25.21,2021-01-01 03:00:00
4,25.53,24.68,25.10,2021-01-01 04:00:00
...,...,...,...,...
17515,24.83,16.94,20.89,2022-12-31 19:00:00
17516,24.73,16.83,20.78,2022-12-31 20:00:00
17517,24.69,16.70,20.69,2022-12-31 21:00:00
17518,24.73,16.55,20.64,2022-12-31 22:00:00


In [ ]:
# convert the time columns in both tables to datetime format
df['Time'] = pd.to_datetime(df['Time'])
temp['datetime'] = pd.to_datetime(temp['datetime'])

# merge the tables on the Time column from df and the datetime column from temp
merged_table = pd.merge(df, temp, left_on='Time', right_on='datetime', how='inner')

# drop the datetime column from the merged table
merged_table.drop('datetime', axis=1, inplace=True)

merged_table

,Time,system_kW,T2M,T2MDEW,T2MWET
0,2021-01-01 00:00:00,9.716,26.11,25.01,25.56
1,2021-01-01 01:00:00,10.959,25.95,24.92,25.44
2,2021-01-01 02:00:00,9.644,25.80,24.86,25.33
3,2021-01-01 03:00:00,9.644,25.65,24.77,25.21
4,2021-01-01 04:00:00,9.754,25.53,24.68,25.10
...,...,...,...,...,...
14510,2022-12-31 19:00:00,10.274,24.83,16.94,20.89
14511,2022-12-31 20:00:00,10.099,24.73,16.83,20.78
14512,2022-12-31 21:00:00,10.962,24.69,16.70,20.69
14513,2022-12-31 22:00:00,10.888,24.73,16.55,20.64


In [ ]:
df = merged_table[['Time','system_kW','T2M']]
df

,Time,system_kW,T2M
0,2021-01-01 00:00:00,9.716,26.11
1,2021-01-01 01:00:00,10.959,25.95
2,2021-01-01 02:00:00,9.644,25.80
3,2021-01-01 03:00:00,9.644,25.65
4,2021-01-01 04:00:00,9.754,25.53
...,...,...,...
14510,2022-12-31 19:00:00,10.274,24.83
14511,2022-12-31 20:00:00,10.099,24.73
14512,2022-12-31 21:00:00,10.962,24.69
14513,2022-12-31 22:00:00,10.888,24.73


In [ ]:
df['hour_of_the_week'] = df['Time'].dt.dayofweek * 24 + (df['Time'].dt.hour + 1)
df['hour_of_the_month'] = (df['Time'].dt.day - 1) * 24 + (df['Time'].dt.hour + 1)
df['hour_of_the_year'] = (df['Time'] - pd.Timestamp(df['Time'].dt.year[0], 1, 1)).dt.total_seconds() // 3600 + 1
df['hour'] = df['Time'].dt.hour
df

,Time,system_kW,T2M,hour_of_the_week,hour_of_the_month,hour_of_the_year,hour
0,2021-01-01 00:00:00,9.716,26.11,97,1,1.0,0
1,2021-01-01 01:00:00,10.959,25.95,98,2,2.0,1
2,2021-01-01 02:00:00,9.644,25.80,99,3,3.0,2
3,2021-01-01 03:00:00,9.644,25.65,100,4,4.0,3
4,2021-01-01 04:00:00,9.754,25.53,101,5,5.0,4
...,...,...,...,...,...,...,...
14510,2022-12-31 19:00:00,10.274,24.83,140,740,17516.0,19
14511,2022-12-31 20:00:00,10.099,24.73,141,741,17517.0,20
14512,2022-12-31 21:00:00,10.962,24.69,142,742,17518.0,21
14513,2022-12-31 22:00:00,10.888,24.73,143,743,17519.0,22


In [ ]:
df[df['hour_of_the_week']==168]

,Time,system_kW,T2M,hour_of_the_week,hour_of_the_month,hour_of_the_year,hour
71,2021-01-03 23:00:00,11.062,26.51,168,72,72.0,23
239,2021-01-10 23:00:00,9.429,25.85,168,240,240.0,23
405,2021-01-17 23:00:00,10.811,25.87,168,408,408.0,23
573,2021-01-24 23:00:00,11.157,24.55,168,576,576.0,23
741,2021-01-31 23:00:00,13.250,25.94,168,744,744.0,23
...,...,...,...,...,...,...,...
13699,2022-11-27 23:00:00,9.218,26.10,168,648,16704.0,23
13867,2022-12-04 23:00:00,12.965,26.19,168,96,16872.0,23
14035,2022-12-11 23:00:00,9.942,25.51,168,264,17040.0,23
14202,2022-12-18 23:00:00,9.394,24.97,168,432,17208.0,23


In [ ]:
# Time	system_kW	T2M
# 2021-01-01 00:00:00	9.716	26.11
# 2021-01-01 01:00:00	10.959	25.95
# 2021-01-01 02:00:00	9.644	25.80
# 2021-01-01 03:00:00	9.644	25.65
# 2021-01-01 04:00:00	9.754	25.53

## Using OLS Regression (Hour vs T2M)

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Load data
df =df

# add hour of the day as a new variable
df['hour'] = df['Time'].dt.hour

# perform linear regression between system_kW, T2M, and hour
X = sm.add_constant(df[['T2M', 'hour']])
model = sm.OLS(df['system_kW'], X).fit()

# print the regression summary
print(model.summary())

# the slope of the T2M coefficient gives the change in system_kW for a one-degree change in T2M
print('Change in system_kW for a one-degree change in T2M: {:.3f}'.format(model.params['T2M']))

# make predictions on the training set
df['predicted'] = model.predict(X)

# calculate mean absolute error, mean squared error, and root mean squared error
mae_train = np.mean(np.abs(df['predicted'] - df['system_kW']))
mse_train = np.mean((df['predicted'] - df['system_kW']) ** 2)
rmse_train = np.sqrt(mse_train)

print('Training set performance:')
print('MAE: {:.2f}'.format(mae_train))
print('MSE: {:.2f}'.format(mse_train))
print('RMSE: {:.2f}'.format(rmse_train))
# the slope of the T2M coefficient gives the change in system_kW for a one-degree change in T2M
print('Change in system_kW for a one-degree change in T2M: {:.3f}'.format(model.params['T2M']))

                            OLS Regression Results                            
Dep. Variable:              system_kW   R-squared:                       0.332
Model:                            OLS   Adj. R-squared:                  0.332
Method:                 Least Squares   F-statistic:                     3609.
Date:                Fri, 24 Mar 2023   Prob (F-statistic):               0.00
Time:                        13:20:55   Log-Likelihood:                -52463.
No. Observations:               14515   AIC:                         1.049e+05
Df Residuals:                   14512   BIC:                         1.050e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -78.3624      1.128    -69.494      0.0

The OLS regression results provide information about how well the model fits the data and the relationships between the variables. Here's how to interpret the various parts of the output:

* R-squared: This value indicates the proportion of variation in the dependent variable (system_kW) that is explained by the independent variables (T2M and hour) in the model. In this case, the R-squared value is 0.332, which means that the model explains about 33.2% of the variation in the system_kW variable.

* Coefficients: The coefficients show the estimated effect of each independent variable on the dependent variable, holding all other variables constant. In this model, the intercept (const) is -78.3624, which means that if the T2M and hour values are both zero, the expected value of system_kW is -78.3624. The coefficient for T2M is 3.6066, which means that a one-unit increase in T2M is associated with a 3.6066 unit increase in system_kW, holding hour constant. The coefficient for hour is -0.0290, which means that a one-unit increase in hour is associated with a 0.0290 unit decrease in system_kW, holding T2M constant.

* Standard errors and t-statistics: These values are used to test the statistical significance of the coefficients. The t-statistic for each coefficient is calculated as the coefficient divided by its standard error. The p-value associated with each t-statistic indicates the probability of obtaining a value as extreme or more extreme than the observed value, assuming the null hypothesis that the true coefficient is zero. In this case, both T2M and hour have very small p-values (less than 0.01), which means that they are statistically significant predictors of system_kW in the model.

* Other statistics: The output also includes other statistics such as the F-statistic, which tests the overall significance of the model, and the AIC and BIC, which are measures of model fit that take into account the number of parameters in the model. The Omnibus and Jarque-Bera tests are tests of normality for the residuals of the model, and the Durbin-Watson test is a test for autocorrelation in the residuals.




In [ ]:
fig = px.line(df.reset_index(), x='Time', y=['system_kW','predicted'], title = 'Time VS Load')

fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt

# # create a pandas DataFrame with the given data
# data =df

# # plot the data
# fig, ax = plt.subplots()
# ax.plot(data['Time'], data['system_kW'], label='System kW')
# ax.plot(data['Time'], data['T2M'], label='Temperature (°C)')
# ax.set_xlabel('Time')
# ax.set_ylabel('Value')
# ax.set_title('System kW and Temperature over Time')
# ax.legend()

# plt.show()


In [ ]:
df.to_csv('full2022-2022.csv',index=False)

There are several other statistical tests that can be used to compute an analysis similar to OLS regression. Some examples include:

1. **Ridge regression**: This is a regularization technique that can be used to prevent overfitting in a regression model by adding a penalty term to the least squares objective function. Ridge regression can be particularly useful when dealing with multicollinearity (high correlation between predictor variables) in the data.

2. **Lasso regression**: Lasso regression is another regularization technique that can be used to select a subset of relevant predictor variables in a regression model. Lasso regression adds a penalty term to the least squares objective function, but unlike ridge regression, it can shrink some of the coefficient estimates to exactly zero, effectively removing those variables from the model.

3. **Elastic net regression**: Elastic net regression is a combination of ridge and lasso regression that can be used to handle both multicollinearity and variable selection in a regression model. Elastic net regression adds both L1 (lasso) and L2 (ridge) penalty terms to the objective function, and allows for tuning of the balance between these penalties.

4. **Principal component regression**: Principal component regression (PCR) is a technique that can be used to reduce the dimensionality of the predictor variables in a regression model by transforming them into a smaller set of uncorrelated variables. PCR can be useful when dealing with high-dimensional data or when the predictor variables are highly correlated.

5. **Partial least squares regression**: Partial least squares (PLS) regression is a technique that is similar to PCR, but instead of creating uncorrelated variables, it creates a smaller set of variables that are linear combinations of the original predictor variables and are highly correlated with the response variable. PLS regression can be useful when there is a strong linear relationship between the predictor and response variables, but the predictor variables are highly correlated with each other.

# USING ARIMA

For time series data, autoregressive integrated moving average (ARIMA) and its variations, such as seasonal ARIMA (SARIMA), are commonly used. These models take into account the time series structure of the data, such as trend, seasonality, and autocorrelation, and can be used to make predictions and forecast future values. Other time series models that may be suitable depending on the specific characteristics of the data include exponential smoothing models, state space models, and dynamic linear models.

###### *T2M*

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Load data
data = df

# Create a differenced series to remove trend and seasonality
diff_data = data.diff().dropna()

# Fit an ARIMA model
model = sm.tsa.ARIMA(diff_data['system_kW'], order=(1,0,0), exog=diff_data['T2M']).fit()

# Print model summary
print(model.summary())



# Predict values for training set
y_train_pred = model.predict()
data['predicted'] = np.nan
data['predicted']= data['system_kW'][0:-1] + y_train_pred.cumsum()

# Calculate MAE, MSE, and RMSE for training set
mae_train = np.mean(np.abs(y_train_pred - diff_data['system_kW']))
mse_train = np.mean(np.square(y_train_pred - diff_data['system_kW']))
rmse_train = np.sqrt(mse_train)

print(f"MAE (training set): {mae_train:.4f}")
print(f"MSE (training set): {mse_train:.4f}")
print(f"RMSE (training set): {rmse_train:.4f}")
data['predicted'] = y_train_pred

                               SARIMAX Results                                
Dep. Variable:              system_kW   No. Observations:                14514
Model:                 ARIMA(1, 0, 0)   Log Likelihood              -41968.554
Date:                Fri, 24 Mar 2023   AIC                          83945.108
Time:                        13:20:58   BIC                          83975.439
Sample:                             0   HQIC                         83955.188
                              - 14514                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0004      0.038      0.010      0.992      -0.074       0.074
T2M            4.0760      0.062     65.523      0.000       3.954       4.198
ar.L1          0.0062      0.006      1.094      0.2

In [ ]:
# Calculate effect of a 1 degree change in temperature on load curve
coef_T2M = model.params['T2M']
print(f"A 1 degree change in temperature leads to a {coef_T2M:.2f} kW change in the load curve.")


A 1 degree change in temperature leads to a 4.08 kW change in the load curve.


* The SARIMAX model used has an order of (1,0,0) indicating an ARIMA model with a
first-order autoregressive term.
* The model shows a positive coefficient of 4.0760 for the T2M variable indicating that a one-degree change in T2M results in a 4.0760 unit change in system_kW.
* The p-value for the T2M variable is very low, indicating that it is a statistically significant predictor of system_kW.
* The coefficient for the autoregressive term (ar.L1) is very low at 0.0062, and its p-value is not significant, indicating that there is no significant autocorrelation in the residuals of the model.
* The model has a high value of sigma2 (19.0168), indicating that there is a relatively high variance in the errors of the model.
* The Ljung-Box test for autocorrelation of the residuals at lag 1 has a p-value of 1.00, indicating that there is no significant autocorrelation in the residuals.
* The model has a low heteroskedasticity (H) value of 0.71, indicating that there is no significant difference in the variances of the residuals across the range of the data.

In [ ]:
import matplotlib.pyplot as plt

# Plot actual vs predicted values
plt.figure(figsize=(10,5))
plt.plot(diff_data.index, diff_data['system_kW'], label='Actual')
plt.plot(diff_data.index, y_train_pred, label='Predicted')
plt.legend()
plt.title('Actual vs Predicted (Training Set)')
plt.xlabel('Date')
plt.ylabel('System kW')
plt.show()

# Plot residuals
residuals = model.resid
plt.figure(figsize=(10,5))
plt.plot(diff_data.index, residuals)
plt.title('Residuals (Training Set)')
plt.xlabel('Date')
plt.ylabel('Residuals')
plt.show()


######'T2M', 'hour'

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Load data
data = df


# Create a differenced series to remove trend and seasonality
diff_data = data.diff().dropna()

# Fit an ARIMA model
model = sm.tsa.ARIMA(diff_data['system_kW'], order=(1,0,0), exog=diff_data[['T2M', 'hour']]).fit()

# Print model summary
print(model.summary())

# Predict values for training set
y_train_pred = model.predict()

# Calculate MAE, MSE, and RMSE for training set
mae_train = np.mean(np.abs(y_train_pred - diff_data['system_kW']))
mse_train = np.mean(np.square(y_train_pred - diff_data['system_kW']))
rmse_train = np.sqrt(mse_train)

print(f"MAE (training set): {mae_train:.4f}")
print(f"MSE (training set): {mse_train:.4f}")
print(f"RMSE (training set): {rmse_train:.4f}")

# Calculate effect of a 1 degree change in temperature on load curve
coef_T2M = model.params['T2M']
print(f"A 1 degree change in temperature leads to a {coef_T2M:.2f} kW change in the load curve.")


/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


                               SARIMAX Results                                
Dep. Variable:              system_kW   No. Observations:                14514
Model:                 ARIMA(1, 0, 0)   Log Likelihood              -41965.415
Date:                Fri, 24 Mar 2023   AIC                          83940.829
Time:                        10:02:57   BIC                          83978.743
Sample:                             0   HQIC                         83953.429
                              - 14514                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0004      0.050      0.008      0.994      -0.097       0.098
T2M            4.0883      0.062     65.842      0.000       3.967       4.210
hour          -0.0189      0.034     -0.561      0.5

* The model uses an ARIMA(1, 0, 0) structure, meaning it includes an autoregressive term with a lag of 1 and no moving average terms.
* The log likelihood of the model is -41965.415, and the AIC (Akaike Information Criterion) is 83940.829.
* The model includes three predictor variables: T2M (temperature in degrees Celsius), hour (time of day), and a constant term.
* The coefficient for T2M is 4.0883, indicating that an increase in temperature is associated with an increase in system_kW.
* The coefficient for hour is -0.0189, but it is not statistically significant at the 0.05 level.
* The coefficient for the autoregressive term (ar.L1) is 0.0052 and is also not statistically significant at the 0.05 level.
* The estimated variance of the error term (sigma2) is 19.0091.
* The Ljung-Box statistic (Q) for the first lag is 0.00, indicating no significant autocorrelation in the residuals.
* The Jarque-Bera statistic (JB) is 54103.42, and the probability of obtaining a JB statistic as extreme as this value is effectively zero. This indicates that the distribution of the residuals is not normal.
* The model exhibits heteroskedasticity, with a value of 0.71 for the Heteroskedasticity statistic (H).






###### Hour of week , T2M

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Load data
data = df


# Create a differenced series to remove trend and seasonality
diff_data = data.diff().dropna()

# Fit an ARIMA model
model = sm.tsa.ARIMA(diff_data['system_kW'], order=(1,0,0), exog=diff_data[['T2M', 'hour_of_the_week']]).fit()

# Print model summary
print(model.summary())

# Predict values for training set
y_train_pred = model.predict()

# Calculate MAE, MSE, and RMSE for training set
mae_train = np.mean(np.abs(y_train_pred - diff_data['system_kW']))
mse_train = np.mean(np.square(y_train_pred - diff_data['system_kW']))
rmse_train = np.sqrt(mse_train)

print(f"MAE (training set): {mae_train:.4f}")
print(f"MSE (training set): {mse_train:.4f}")
print(f"RMSE (training set): {rmse_train:.4f}")

# Calculate effect of a 1 degree change in temperature on load curve
coef_T2M = model.params['T2M']
print(f"A 1 degree change in temperature leads to a {coef_T2M:.2f} kW change in the load curve.")


/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


                               SARIMAX Results                                
Dep. Variable:              system_kW   No. Observations:                14514
Model:                 ARIMA(1, 0, 0)   Log Likelihood              -41967.461
Date:                Fri, 24 Mar 2023   AIC                          83944.923
Time:                        10:06:31   BIC                          83982.837
Sample:                             0   HQIC                         83957.523
                              - 14514                                         
Covariance Type:                  opg                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.0004      0.039      0.010      0.992      -0.075       0.076
T2M                  4.0783      0.062     65.585      0.000       3.956       4.200
hour_of_the_week    -0.0040 

###### Week in Month, T2M

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Load data
data = df


# Create a differenced series to remove trend and seasonality
diff_data = data.diff().dropna()

# Fit an ARIMA model
model = sm.tsa.ARIMA(diff_data['system_kW'], order=(1,0,0), exog=diff_data[['T2M', 'hour_of_the_month']]).fit()

# Print model summary
print(model.summary())

# Predict values for training set
y_train_pred = model.predict()

# Calculate MAE, MSE, and RMSE for training set
mae_train = np.mean(np.abs(y_train_pred - diff_data['system_kW']))
mse_train = np.mean(np.square(y_train_pred - diff_data['system_kW']))
rmse_train = np.sqrt(mse_train)

print(f"MAE (training set): {mae_train:.4f}")
print(f"MSE (training set): {mse_train:.4f}")
print(f"RMSE (training set): {rmse_train:.4f}")

# Calculate effect of a 1 degree change in temperature on load curve
coef_T2M = model.params['T2M']
print(f"A 1 degree change in temperature leads to a {coef_T2M:.2f} kW change in the load curve.")


/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


                               SARIMAX Results                                
Dep. Variable:              system_kW   No. Observations:                14514
Model:                 ARIMA(1, 0, 0)   Log Likelihood              -41968.490
Date:                Fri, 24 Mar 2023   AIC                          83946.980
Time:                        10:08:41   BIC                          83984.894
Sample:                             0   HQIC                         83959.580
                              - 14514                                         
Covariance Type:                  opg                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.0004      0.038      0.010      0.992      -0.074       0.075
T2M                   4.0764      0.062     65.531      0.000       3.954       4.198
hour_of_the_month    -0.

###### hour_of_the_year, T2M

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Load data
data = df
# Create a differenced series to remove trend and seasonality
diff_data = data.diff().dropna()

# Fit an ARIMA model
model = sm.tsa.ARIMA(diff_data['system_kW'], order=(1,0,0), exog=diff_data[['T2M', 'hour_of_the_year']]).fit()

# Print model summary
print(model.summary())

# Predict values for training set
y_train_pred = model.predict()

# Calculate MAE, MSE, and RMSE for training set
mae_train = np.mean(np.abs(y_train_pred - diff_data['system_kW']))
mse_train = np.mean(np.square(y_train_pred - diff_data['system_kW']))
rmse_train = np.sqrt(mse_train)

print(f"MAE (training set): {mae_train:.4f}")
print(f"MSE (training set): {mse_train:.4f}")
print(f"RMSE (training set): {rmse_train:.4f}")


/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


                               SARIMAX Results                                
Dep. Variable:              system_kW   No. Observations:                14514
Model:                 ARIMA(1, 0, 0)   Log Likelihood              -41967.514
Date:                Fri, 24 Mar 2023   AIC                          83945.028
Time:                        10:12:49   BIC                          83982.942
Sample:                             0   HQIC                         83957.627
                              - 14514                                         
Covariance Type:                  opg                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.0087      0.041     -0.215      0.830      -0.088       0.071
T2M                  4.0758      0.062     65.497      0.000       3.954       4.198
hour_of_the_year     0.0076 

Day of Week

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Load data
data = df

# Create day of the week column
data['day_of_week'] = pd.to_datetime(data['Time']).dt.dayofweek

# Create a differenced series to remove trend and seasonality
diff_data = data.diff().dropna()

# Fit an ARIMA model
model = sm.tsa.ARIMA(diff_data['system_kW'], order=(1,0,0), exog=diff_data[['T2M', 'day_of_week']]).fit()

# Print model summary
print(model.summary())

# Predict values for training set
y_train_pred = model.predict()

# Calculate MAE, MSE, and RMSE for training set
mae_train = np.mean(np.abs(y_train_pred - diff_data['system_kW']))
mse_train = np.mean(np.square(y_train_pred - diff_data['system_kW']))
rmse_train = np.sqrt(mse_train)

print(f"MAE (training set): {mae_train:.4f}")
print(f"MSE (training set): {mse_train:.4f}")
print(f"RMSE (training set): {rmse_train:.4f}")


/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


                               SARIMAX Results                                
Dep. Variable:              system_kW   No. Observations:                14514
Model:                 ARIMA(1, 0, 0)   Log Likelihood              -41968.358
Date:                Fri, 24 Mar 2023   AIC                          83946.717
Time:                        10:38:22   BIC                          83984.631
Sample:                             0   HQIC                         83959.317
                              - 14514                                         
Covariance Type:                  opg                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.0004      0.038      0.010      0.992      -0.074       0.074
T2M             4.0759      0.062     65.518      0.000       3.954       4.198
day_of_week    -0.0431      0.212     -0.203    

In [ ]:
# temperature by building and by industry.
# reference load plays.
y = 0.0004 + 4.076T2M - 0.0431DOW + 19.0162

In [ ]:
data

,Time,system_kW,T2M,hour_of_the_week,hour_of_the_month,hour_of_the_year,hour,day_of_week
0,2021-01-01 00:00:00,9.716,26.11,97,0,1.0,0,4
1,2021-01-01 01:00:00,10.959,25.95,98,1,2.0,1,4
2,2021-01-01 02:00:00,9.644,25.80,99,2,3.0,2,4
3,2021-01-01 03:00:00,9.644,25.65,100,3,4.0,3,4
4,2021-01-01 04:00:00,9.754,25.53,101,4,5.0,4,4
...,...,...,...,...,...,...,...,...
14510,2022-12-31 19:00:00,10.274,24.83,140,19,17516.0,19,5
14511,2022-12-31 20:00:00,10.099,24.73,141,20,17517.0,20,5
14512,2022-12-31 21:00:00,10.962,24.69,142,21,17518.0,21,5
14513,2022-12-31 22:00:00,10.888,24.73,143,22,17519.0,22,5


In [ ]:
df

,system_kW,T2M,hour_of_the_week,hour_of_the_month,hour_of_the_year,hour,day_of_week
Time,,,,,,,
2021-01-01 00:00:00,9.716,26.11,97,0,1.0,0,4
2021-01-01 01:00:00,10.959,25.95,98,1,2.0,1,4
2021-01-01 02:00:00,9.644,25.80,99,2,3.0,2,4
2021-01-01 03:00:00,9.644,25.65,100,3,4.0,3,4
2021-01-01 04:00:00,9.754,25.53,101,4,5.0,4,4
...,...,...,...,...,...,...,...
2022-12-31 19:00:00,10.274,24.83,140,19,17516.0,19,5
2022-12-31 20:00:00,10.099,24.73,141,20,17517.0,20,5
2022-12-31 21:00:00,10.962,24.69,142,21,17518.0,21,5


In [ ]:
# df['Time'] = pd.to_datetime(df['Time'])
df.reset_index(inplace=True)
df = df.set_index('Time')

# Filter the DataFrame to only include data between 8am and 5pm
filtered_df = df.between_time('08:00', '17:00')
# Convert index to datetime
filtered_df.index = pd.to_datetime(filtered_df.index)

# Remove Saturdays and Sundays
filtered_df = filtered_df[(filtered_df.index.weekday != 5) & (filtered_df.index.weekday != 6)]


WORKING HOURS

In [ ]:
filtered_df = filtered_df.reset_index()
filtered_df

,Time,system_kW,T2M,hour_of_the_week,hour_of_the_month,hour_of_the_year,hour,day_of_week
0,2021-01-01 08:00:00,10.703,27.46,105,8,9.0,8,4
1,2021-01-01 09:00:00,9.543,28.16,106,9,10.0,9,4
2,2021-01-01 10:00:00,9.566,28.70,107,10,11.0,10,4
3,2021-01-01 11:00:00,9.561,29.07,108,11,12.0,11,4
4,2021-01-01 12:00:00,9.835,29.28,109,12,13.0,12,4
...,...,...,...,...,...,...,...,...
4329,2022-12-30 13:00:00,29.127,29.63,110,13,17486.0,13,4
4330,2022-12-30 14:00:00,25.695,29.43,111,14,17487.0,14,4
4331,2022-12-30 15:00:00,28.754,29.11,112,15,17488.0,15,4
4332,2022-12-30 16:00:00,21.636,28.12,113,16,17489.0,16,4


In [ ]:
import pandas as pd
import statsmodels.api as sm

df = filtered_df
# add hour of the day as a new variable
df['hour'] = df['Time'].dt.hour

# perform linear regression between system_kW, T2M, and hour
X = sm.add_constant(df[['T2M', 'hour']])
model = sm.OLS(df['system_kW'], X).fit()

# print the regression summary
print(model.summary())

# the slope of the T2M coefficient gives the change in system_kW for a one-degree change in T2M
print('Change in system_kW for a one-degree change in T2M: {:.3f}'.format(model.params['T2M']))



# make predictions on the training set
y_train_pred = model.predict(X)

# calculate mean absolute error, mean squared error, and root mean squared error
mae_train = np.mean(np.abs(y_train_pred - df['system_kW']))
mse_train = np.mean((y_train_pred - df['system_kW']) ** 2)
rmse_train = np.sqrt(mse_train)

print('Training set performance:')
print('MAE: {:.2f}'.format(mae_train))
print('MSE: {:.2f}'.format(mse_train))
print('RMSE: {:.2f}'.format(rmse_train))

                            OLS Regression Results                            
Dep. Variable:              system_kW   R-squared:                       0.214
Model:                            OLS   Adj. R-squared:                  0.214
Method:                 Least Squares   F-statistic:                     590.3
Date:                Fri, 24 Mar 2023   Prob (F-statistic):          1.94e-227
Time:                        11:03:53   Log-Likelihood:                -15083.
No. Observations:                4334   AIC:                         3.017e+04
Df Residuals:                    4331   BIC:                         3.019e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -54.4902      2.502    -21.782      0.0

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Load data
data = filtered_df

# Create a differenced series to remove trend and seasonality
diff_data = data.diff().dropna()

# Fit an ARIMA model
model = sm.tsa.ARIMA(diff_data['system_kW'], order=(1,0,0), exog=diff_data['T2M']).fit()

# Print model summary
print(model.summary())



# Predict values for training set
y_train_pred = model.predict()

# Calculate MAE, MSE, and RMSE for training set
mae_train = np.mean(np.abs(y_train_pred - diff_data['system_kW']))
mse_train = np.mean(np.square(y_train_pred - diff_data['system_kW']))
rmse_train = np.sqrt(mse_train)

print(f"MAE (training set): {mae_train:.4f}")
print(f"MSE (training set): {mse_train:.4f}")
print(f"RMSE (training set): {rmse_train:.4f}")



import pandas as pd
import numpy as np
import statsmodels.api as sm

# Load data
data = filtered_df

# Create a differenced series to remove trend and seasonality
diff_data = data.diff().dropna()

# Fit an ARIMA model
model = sm.tsa.ARIMA(diff_data['system_kW'], order=(1,0,0), exog=diff_data['T2M']).fit()

# Print model summary
print(model.summary())



# Predict values for training set
y_train_pred = model.predict()

# Calculate MAE, MSE, and RMSE for training set
mae_train = np.mean(np.abs(y_train_pred - diff_data['system_kW']))
mse_train = np.mean(np.square(y_train_pred - diff_data['system_kW']))
rmse_train = np.sqrt(mse_train)

print(f"MAE (training set): {mae_train:.4f}")
print(f"MSE (training set): {mse_train:.4f}")
print(f"RMSE (training set): {rmse_train:.4f}")



import pandas as pd
import numpy as np
import statsmodels.api as sm

# Load data
data = filtered_df


# Create a differenced series to remove trend and seasonality
diff_data = data.diff().dropna()

# Fit an ARIMA model
model = sm.tsa.ARIMA(diff_data['system_kW'], order=(1,0,0), exog=diff_data[['T2M', 'hour_of_the_week']]).fit()

# Print model summary
print(model.summary())

# Predict values for training set
y_train_pred = model.predict()

# Calculate MAE, MSE, and RMSE for training set
mae_train = np.mean(np.abs(y_train_pred - diff_data['system_kW']))
mse_train = np.mean(np.square(y_train_pred - diff_data['system_kW']))
rmse_train = np.sqrt(mse_train)

print(f"MAE (training set): {mae_train:.4f}")
print(f"MSE (training set): {mse_train:.4f}")
print(f"RMSE (training set): {rmse_train:.4f}")

# Calculate effect of a 1 degree change in temperature on load curve
coef_T2M = model.params['T2M']
print(f"A 1 degree change in temperature leads to a {coef_T2M:.2f} kW change in the load curve.")




import pandas as pd
import numpy as np
import statsmodels.api as sm

# Load data
data = filtered_df


# Create a differenced series to remove trend and seasonality
diff_data = data.diff().dropna()

# Fit an ARIMA model
model = sm.tsa.ARIMA(diff_data['system_kW'], order=(1,0,0), exog=diff_data[['T2M', 'hour_of_the_month']]).fit()

# Print model summary
print(model.summary())

# Predict values for training set
y_train_pred = model.predict()

# Calculate MAE, MSE, and RMSE for training set
mae_train = np.mean(np.abs(y_train_pred - diff_data['system_kW']))
mse_train = np.mean(np.square(y_train_pred - diff_data['system_kW']))
rmse_train = np.sqrt(mse_train)

print(f"MAE (training set): {mae_train:.4f}")
print(f"MSE (training set): {mse_train:.4f}")
print(f"RMSE (training set): {rmse_train:.4f}")

# Calculate effect of a 1 degree change in temperature on load curve
coef_T2M = model.params['T2M']
print(f"A 1 degree change in temperature leads to a {coef_T2M:.2f} kW change in the load curve.")


import pandas as pd
import numpy as np
import statsmodels.api as sm

# Load data
data = filtered_df
# Create a differenced series to remove trend and seasonality
diff_data = data.diff().dropna()

# Fit an ARIMA model
model = sm.tsa.ARIMA(diff_data['system_kW'], order=(1,0,0), exog=diff_data[['T2M', 'hour_of_the_year']]).fit()

# Print model summary
print(model.summary())

# Predict values for training set
y_train_pred = model.predict()

# Calculate MAE, MSE, and RMSE for training set
mae_train = np.mean(np.abs(y_train_pred - diff_data['system_kW']))
mse_train = np.mean(np.square(y_train_pred - diff_data['system_kW']))
rmse_train = np.sqrt(mse_train)

print(f"MAE (training set): {mae_train:.4f}")
print(f"MSE (training set): {mse_train:.4f}")
print(f"RMSE (training set): {rmse_train:.4f}")


import pandas as pd
import numpy as np
import statsmodels.api as sm

# Load data
data = filtered_df

# Create day of the week column
data['day_of_week'] = pd.to_datetime(data['Time']).dt.dayofweek

# Create a differenced series to remove trend and seasonality
diff_data = data.diff().dropna()

# Fit an ARIMA model
model = sm.tsa.ARIMA(diff_data['system_kW'], order=(1,0,0), exog=diff_data[['T2M', 'day_of_week']]).fit()

# Print model summary
print(model.summary())

# Predict values for training set
y_train_pred = model.predict()

# Calculate MAE, MSE, and RMSE for training set
mae_train = np.mean(np.abs(y_train_pred - diff_data['system_kW']))
mse_train = np.mean(np.square(y_train_pred - diff_data['system_kW']))
rmse_train = np.sqrt(mse_train)

print(f"MAE (training set): {mae_train:.4f}")
print(f"MSE (training set): {mse_train:.4f}")
print(f"RMSE (training set): {rmse_train:.4f}")




/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


                               SARIMAX Results                                
Dep. Variable:              system_kW   No. Observations:                 4333
Model:                 ARIMA(1, 0, 0)   Log Likelihood              -13540.404
Date:                Fri, 24 Mar 2023   AIC                          27088.809
Time:                        11:08:57   BIC                          27114.305
Sample:                             0   HQIC                         27097.810
                               - 4333                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0028      0.068      0.042      0.967      -0.131       0.136
T2M            1.8413      0.096     19.224      0.000       1.654       2.029
ar.L1         -0.3098      0.009    -33.331      0.0

/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


                               SARIMAX Results                                
Dep. Variable:              system_kW   No. Observations:                 4333
Model:                 ARIMA(1, 0, 0)   Log Likelihood              -13540.404
Date:                Fri, 24 Mar 2023   AIC                          27088.809
Time:                        11:08:58   BIC                          27114.305
Sample:                             0   HQIC                         27097.810
                               - 4333                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0028      0.068      0.042      0.967      -0.131       0.136
T2M            1.8413      0.096     19.224      0.000       1.654       2.029
ar.L1         -0.3098      0.009    -33.331      0.0

/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


...

                               SARIMAX Results                                
Dep. Variable:              system_kW   No. Observations:                 4333
Model:                 ARIMA(1, 0, 0)   Log Likelihood              -13533.155
Date:                Fri, 24 Mar 2023   AIC                          27076.311
Time:                        11:09:01   BIC                          27108.181
Sample:                             0   HQIC                         27087.562
                               - 4333                                         
Covariance Type:                  opg                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.0027      0.077      0.035      0.972      -0.149       0.154
T2M                   1.8036      0.095     19.077      0.000       1.618       1.989
hour_of_the_month     0.

/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


                               SARIMAX Results                                
Dep. Variable:              system_kW   No. Observations:                 4333
Model:                 ARIMA(1, 0, 0)   Log Likelihood              -13540.298
Date:                Fri, 24 Mar 2023   AIC                          27090.595
Time:                        11:09:02   BIC                          27122.466
Sample:                             0   HQIC                         27101.847
                               - 4333                                         
Covariance Type:                  opg                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.0120      0.075      0.161      0.872      -0.134       0.158
T2M                  1.8402      0.096     19.233      0.000       1.653       2.028
hour_of_the_year    -0.0023 

/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


                               SARIMAX Results                                
Dep. Variable:              system_kW   No. Observations:                 4333
Model:                 ARIMA(1, 0, 0)   Log Likelihood              -13538.322
Date:                Fri, 24 Mar 2023   AIC                          27086.644
Time:                        11:09:04   BIC                          27118.515
Sample:                             0   HQIC                         27097.896
                               - 4333                                         
Covariance Type:                  opg                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.0029      0.068      0.043      0.966      -0.131       0.137
T2M             1.8400      0.096     19.111      0.000       1.651       2.029
day_of_week    -0.2546      0.064     -3.982    

DECISION TREES

In [ ]:
df = pd.read_csv('full2022-2022.csv')
df

,Time,system_kW,T2M,hour_of_the_week,hour_of_the_month,hour_of_the_year,hour
0,2021-01-01 00:00:00,9.716,26.11,97,1,1.0,0
1,2021-01-01 01:00:00,10.959,25.95,98,2,2.0,1
2,2021-01-01 02:00:00,9.644,25.80,99,3,3.0,2
3,2021-01-01 03:00:00,9.644,25.65,100,4,4.0,3
4,2021-01-01 04:00:00,9.754,25.53,101,5,5.0,4
...,...,...,...,...,...,...,...
14510,2022-12-31 19:00:00,10.274,24.83,140,740,17516.0,19
14511,2022-12-31 20:00:00,10.099,24.73,141,741,17517.0,20
14512,2022-12-31 21:00:00,10.962,24.69,142,742,17518.0,21
14513,2022-12-31 22:00:00,10.888,24.73,143,743,17519.0,22


In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load data
data = df

# Add hour column to data frame
data['hour'] = pd.to_datetime(data['Time']).dt.hour

# Create features and target variable
X = data[['T2M']]
y = data['system_kW']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create a decision tree model
model = DecisionTreeRegressor(max_depth=5)

# Train the model on the training set
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate MAE, MSE, and RMSE on the test set
mae_test = mean_absolute_error(y_test, y_pred)
mse_test = mean_squared_error(y_test, y_pred)
rmse_test = np.sqrt(mse_test)
r2_test = r2_score(y_test, y_pred)

print(f"MAE (test set): {mae_test:.4f}")
print(f"MSE (test set): {mse_test:.4f}")
print(f"RMSE (test set): {rmse_test:.4f}")

MAE (test set): 6.4137
MSE (test set): 75.8070
RMSE (test set): 8.7067


RANDOM FOREST

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

data = df

# Convert time column to datetime format
data['Time'] = pd.to_datetime(data['Time'])

# Extract relevant features from time column
data['hour'] = data['Time'].dt.hour
data['day'] = data['Time'].dt.day
data['month'] = data['Time'].dt.month
data['year'] = data['Time'].dt.year

# Define input and output variables
X = data[['T2M', 'hour', 'day', 'month', 'year']]
y = data['system_kW']

# Split data into training and testing sets
train_size = int(len(data) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Fit a random forest regression model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions on test set
y_pred = model.predict(X_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Print evaluation metrics
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")


MAE: 4.5706
MSE: 48.3898
RMSE: 6.9563


AUTO ARIMA ML

In [ ]:
import pandas as pd
import numpy as np
from pmdarima.arima import auto_arima

# Load data
data = df

# Define exogenous variables
exog = data['T2M'].values.reshape(-1, 1)

# Fit an auto ARIMA model with T2M as exogenous variable
model = auto_arima(data['system_kW'], exogenous=exog, seasonal=False)

# Generate predictions for the entire dataset, including the training set and any future values
y_pred = model.predict_in_sample(exogenous=exog)

# Add a new column to the DataFrame containing the predicted values
data['predicted'] = y_pred

# Print model summary
print(model.summary())

# Calculate MAE, MSE, and RMSE for training set
mae_train = np.mean(np.abs(y_pred - data['system_kW']))
mse_train = np.mean(np.square(y_pred - data['system_kW']))
rmse_train = np.sqrt(mse_train)

print(f"MAE (training set): {mae_train:.4f}")
print(f"MSE (training set): {mse_train:.4f}")
print(f"RMSE (training set): {rmse_train:.4f}")


                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                14515
Model:               SARIMAX(0, 1, 3)   Log Likelihood              -43021.333
Date:                Fri, 24 Mar 2023   AIC                          86050.666
Time:                        12:22:17   BIC                          86080.997
Sample:                             0   HQIC                         86060.746
                              - 14515                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.1343      0.005     25.165      0.000       0.124       0.145
ma.L2          0.0875      0.008     10.828      0.000       0.072       0.103
ma.L3          0.0156      0.009      1.823      0.0

In [ ]:
data

,Time,system_kW,T2M,hour_of_the_week,hour_of_the_month,hour_of_the_year,hour,predicted
0,2021-01-01 00:00:00,9.716,26.11,97,1,1.0,0,0.000000
1,2021-01-01 01:00:00,10.959,25.95,98,2,2.0,1,9.716031
2,2021-01-01 02:00:00,9.644,25.80,99,3,3.0,2,11.137669
3,2021-01-01 03:00:00,9.644,25.65,100,4,4.0,3,9.552454
4,2021-01-01 04:00:00,9.754,25.53,101,5,5.0,4,9.545410
...,...,...,...,...,...,...,...,...
14510,2022-12-31 19:00:00,10.274,24.83,140,740,17516.0,19,10.051875
14511,2022-12-31 20:00:00,10.099,24.73,141,741,17517.0,20,10.189716
14512,2022-12-31 21:00:00,10.962,24.69,142,742,17518.0,21,10.087677
14513,2022-12-31 22:00:00,10.888,24.73,143,743,17519.0,22,11.074992


In [ ]:
fig = px.line(data.reset_index(), x='Time', y=['system_kW','predicted'], title = 'Time VS Load')

fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

# calculate mean squared error
mse = mean_squared_error(data['system_kW'], data['predicted'])
print('MSE:', mse)

# calculate root mean squared error
rmse = np.sqrt(mse)
print('RMSE:', rmse)

MSE: 21.990290182111604
RMSE: 4.689380575525045


In [ ]:
# SAVING THE MODEL


import pickle

# save the model to a file
with open('arima_model.pkl', 'wb') as file:
    pickle.dump(model, file)


In [ ]:
import pandas as pd
import numpy as np
from pmdarima.arima import auto_arima
from statsmodels.tsa.stattools import adfuller

# Load data
data =df

# Test for stationarity
adf_result = adfuller(data['system_kW'])
print(f"ADF Statistic: {adf_result[0]:.4f}")
print(f"p-value: {adf_result[1]:.4f}")



ADF Statistic: -16.9771
p-value: 0.0000


Great, the ADF statistic is highly negative and the p-value is less than 0.05, which means we can reject the null hypothesis of non-stationarity and conclude that the system_kW column is stationary. This suggests that an ARIMA model may be appropriate for this data.

In [ ]:
import pandas as pd
import numpy as np
from pmdarima.arima import auto_arima

# Load data
data =df

# Split data into train and test sets
train_size = int(len(data) * 0.8)
train, test = data[:train_size], data[train_size:]

# Fit auto_arima model
arima_model = auto_arima(train['system_kW'],
                         start_p=0, start_q=0, max_p=5, max_d=2, max_q=5,
                         D=1, start_Q=0, max_P=5, max_D=5, max_Q=5, m=12, seasonal=True,
                         error_action='warn', trace=True, suppress_warnings=True, stepwise=True,
                         random_state=20, n_fits=5, exogenous=train['T2M'])

# Print model summary
print(arima_model.summary())


Performing stepwise search to minimize aic
 ARIMA(0,0,0)(1,1,0)[12] intercept   : AIC=85972.594, Time=19.00 sec
 ARIMA(0,0,0)(0,1,0)[12] intercept   : AIC=101118.124, Time=0.89 sec
 ARIMA(1,0,0)(1,1,0)[12] intercept   : AIC=69293.672, Time=21.70 sec
 ARIMA(0,0,1)(0,1,1)[12] intercept   : AIC=inf, Time=35.50 sec
 ARIMA(0,0,0)(0,1,0)[12]             : AIC=101116.124, Time=1.30 sec
 ARIMA(1,0,0)(0,1,0)[12] intercept   : AIC=79221.753, Time=5.20 sec
 ARIMA(1,0,0)(2,1,0)[12] intercept   : AIC=68934.344, Time=55.57 sec
 ARIMA(1,0,0)(3,1,0)[12] intercept   : AIC=67790.739, Time=110.30 sec


KeyboardInterrupt: ignored

In [ ]:
arima_model.summary()


In [ ]:
prediction = pd.DataFrame(arima_model.predict(n_periods=20), index=test.index)
prediction.columns = ['predicted']
prediction

In [ ]:
from sklearn.metrics import r2_score
test['predicted'] = prediction
r2_score(test['Mean_Load'], test['predicted'])